In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests

In [2]:
window=800
infos = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}.pick', "rb"))

In [3]:
infos.to_csv(f'{path_code}/MscThesis/Results/straight_bait_{window}.csv')

In [4]:
infos

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph
0,351,"(35, 0, 15)",25,-44.627955,-0.220995,1.423188,201.941059,50.485265,0.908600,30174.769676,2555.543973,-166.513522,-171.246818,0.0,0
1,351,"(35, 0, 15)",25,18.091350,0.063647,0.925362,284.245706,71.061426,0.956046,29408.835169,3883.177307,178.189595,118.890568,0.0,18
2,351,"(35, 0, 15)",25,59.052703,0.218164,0.765217,270.680056,67.670014,0.910669,29627.815731,3950.743221,126.875173,69.009309,4.0,18
3,351,"(35, 0, 15)",25,-10.236040,-0.040914,0.850000,250.186905,62.546726,0.994030,29827.664022,3954.542957,171.968238,118.122538,12.0,18
4,351,"(35, 0, 15)",25,10.731002,0.033386,0.844203,321.420439,80.355110,0.988017,29968.457318,4013.256159,159.874112,107.556851,16.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8254,424,"(13, 116, 131)",baits,-0.000000,-0.000000,2.710145,76.036890,19.009222,0.925425,32268.647462,2058.679431,14.321099,57.030265,64.0,10105
8255,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,31468.761939,1455.927539,152.677332,-143.438006,60.0,11031
8256,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,31633.890956,1699.400188,134.964020,-166.247137,64.0,11031
8257,424,"(13, 116, 131)",baits,5.017723,0.004998,1.029710,1003.890873,250.972718,1.002913,33495.516431,2850.986671,-128.149699,-110.700223,64.0,11761


In [4]:
corrected = infos.loc[(infos["straightness"] <= 1)&(infos["plate"] !=435)& (infos["speed"] >=25)&(infos["straightness"] > 0.95)]

In [5]:
baits = corrected .loc[corrected ['treatment']=='baits']

In [83]:
no_baits = corrected .loc[corrected ['treatment']=='25']

In [5]:
baits

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph
3913,395,"(10, 0, 10)",baits,6.554181,0.028963,1.000725,226.294882,56.573721,0.990876,32122.539081,13621.389430,-127.810700,158.132921,16.0,99
3914,395,"(10, 0, 10)",baits,-11.309932,-0.030246,0.857971,373.937647,93.484412,0.986906,32208.938464,13772.913744,-124.844503,161.439781,20.0,99
3915,395,"(10, 0, 10)",baits,4.466079,0.008599,0.728261,519.382846,129.845711,0.996801,32323.328572,13977.510866,-125.195068,161.447649,24.0,99
3916,395,"(10, 0, 10)",baits,-4.467159,-0.007155,0.587681,624.316099,156.079025,0.991340,32509.767532,14277.103698,-129.003044,158.792263,28.0,99
3917,395,"(10, 0, 10)",baits,12.260655,0.016484,0.657246,743.776576,185.172591,0.996342,32724.901237,14572.463244,-131.096579,157.346641,32.0,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,424,"(13, 116, 131)",baits,7.281669,0.012852,0.903623,566.586163,141.646541,0.968746,32933.136459,2760.743559,-36.050454,4.413616,56.0,10105
8253,424,"(13, 116, 131)",baits,-20.905807,-0.030679,2.035507,681.432714,170.358178,0.984990,32640.146721,2446.169454,-34.877001,6.786954,60.0,10105
8255,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,31468.761939,1455.927539,152.677332,-143.438006,60.0,11031
8256,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,31633.890956,1699.400188,134.964020,-166.247137,64.0,11031


In [9]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(baits['dist_P'],baits['speed'])
ax.set_ylim(0,400)
# ax.set_xlim(26000,32000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 400)

In [11]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(np.abs(baits['angle_to_P']),baits['speed'])
ax.set_ylim(0,400)
# ax.set_xlim(26000,32000)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 400)

In [72]:
lambd = 10000
baits['gradient']=np.cos(baits['angle_to_P'])*baits['dist_P']*np.exp(-(baits['dist_P']/lambd)**2)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [74]:
baits_sort = baits.sort_values('gradient')

In [57]:
baits_sort.rolling(100,min_periods=1).mean()

,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,gradient
3980,2.826822,0.002160,0.928261,1308.674252,325.811017,0.993061,7208.961437,7208.961437,53.364389,53.364389,32.000000,-4283.293841
6065,-5.824231,-0.008594,1.478623,1028.399366,256.812347,0.985860,7166.236079,7166.236079,50.211813,50.211813,36.000000,-4281.459074
3981,-2.833782,-0.004949,1.312077,1133.397578,283.157731,0.989899,7084.308754,7084.308754,53.343712,53.343712,36.005556,-4278.369633
4478,0.465150,0.000302,1.527899,1011.392250,252.704315,0.991251,7235.756110,7235.756110,10.930175,10.930175,49.833333,-4270.299744
5871,-0.319830,-0.000526,1.906667,989.245393,247.196350,0.990204,7067.064352,7067.064352,25.714969,25.714969,42.263333,-4265.453995
...,...,...,...,...,...,...,...,...,...,...,...,...
4048,2.294570,0.003823,2.812616,914.868609,214.812279,0.987378,7808.249983,7808.249983,-9.470454,-9.470454,49.439667,3532.262652
4406,2.391351,0.003898,2.779841,916.415030,215.187144,0.987354,7832.749936,7832.749936,-9.600941,-9.600941,49.832833,3544.372686
6092,2.295229,0.003737,2.804986,912.169403,214.132514,0.987153,7833.266665,7833.266665,-7.533915,-7.533915,49.274167,3556.576449
5980,2.763804,0.004267,2.806043,916.542541,215.236552,0.986880,7841.021869,7841.021869,-5.013371,-5.013371,48.715333,3569.040237


In [82]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 10000
baits['gradient']=np.cos(baits['angle_to_P'])*baits['dist_P']*np.exp(-(baits['dist_P']/lambd)**2)
baits_sort = baits.sort_values('gradient')
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits["gradient"],baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["gradient"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["gradient"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["gradient"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 100000
baits['gradient']=np.cos(baits['angle_to_P'])*baits['dist_P']*np.exp(-(baits['dist_P']/lambd)**2)
baits_sort = baits.sort_values('dist_P')
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits['dist_P'],baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av['dist_P'],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 100000
no_baits['gradient']=np.cos(no_baits['angle_to_P'])*no_baits['dist_P']*np.exp(-(no_baits['dist_P']/lambd)**2)
baits_sort = no_baits.sort_values('dist_P')
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(no_baits['dist_P'],no_baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av['dist_P'],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
p_value

0.02223930581323196

In [31]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
zipped_list = zip(np.abs(baits['angle_to_P']),baits['straightness'])
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('curvilinear length ($\mu m$')
# ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = 180
# ax.set_xlim(400,1500)
# ax.set_ylim((0.97,1.01))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=100
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
zipped_list = zip(baits['dist_P']*1.725/1000,baits['speed'])
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('distance to P(mm)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 70
ax.set_ylim(0,400)
# ax.set_ylim((0.97,1.01))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=300
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
p_value

0.705453877735688